In [1]:
#using Pkg
#Pkg.activate(".")

In [4]:
using LinearAlgebra
using LimberJack
using CSV
using YAML
using JLD2
using PythonCall
using DataFrames
using NPZ
using Plots
using Interpolations
using Turing
sacc = pyimport("sacc");

In [17]:
method = "sompz"
sacc_path = "data/CosmoDC2/summary_statistics_fourier_tjpcov.sacc"
yaml_path = "data/CosmoDC2/gcgc_gcwl_wlwl.yml"
nz_path = string("data/CosmoDC2/nzs_", method, "/")
param_path = string("data/CosmoDC2/nzs_", method, "/dz_priors/dz_params.npz")

sacc_file = sacc.Sacc().load_fits(sacc_path)
yaml_file = YAML.load_file(yaml_path)

nz_lens_0 = npzread(string(nz_path, "lens_0.npz"))
nz_lens_1 = npzread(string(nz_path, "lens_1.npz"))
nz_lens_2 = npzread(string(nz_path, "lens_2.npz"))
nz_lens_3 = npzread(string(nz_path, "lens_3.npz"))
nz_lens_4 = npzread(string(nz_path, "lens_4.npz"))
nz_source_0 = npzread(string(nz_path, "source_0.npz"))
nz_source_1 = npzread(string(nz_path, "source_1.npz"))
nz_source_2 = npzread(string(nz_path, "source_2.npz"))
nz_source_3 = npzread(string(nz_path, "source_3.npz"))
nz_source_4 = npzread(string(nz_path, "source_4.npz"))


realization = 100
dz_params = npzread(string("data/CosmoDC2/nzs_", method, "/dz_priors/dz_params.npz"))
dz_0 = dz_params["lens_0"][:, realization]
dz_1 = dz_params["lens_1"][:, realization]
dz_2 = dz_params["lens_2"][:, realization]
dz_3 = dz_params["lens_3"][:, realization]
dz_4 = dz_params["lens_4"][:, realization]
dz_5 = dz_params["source_0"][:, realization]
dz_6 = dz_params["source_1"][:, realization]
dz_7 = dz_params["source_2"][:, realization]
dz_8 = dz_params["source_3"][:, realization]
dz_9 = dz_params["source_4"][:, realization]

zs_k0, nz_k0 = nz_lens_0["z"], nz_lens_0["photo_hists"][:, realization]
zs_k1, nz_k1 = nz_lens_1["z"], nz_lens_1["photo_hists"][:, realization]
zs_k2, nz_k2 = nz_lens_2["z"], nz_lens_2["photo_hists"][:, realization]
zs_k3, nz_k3 = nz_lens_3["z"], nz_lens_3["photo_hists"][:, realization]
zs_k4, nz_k4 = nz_lens_4["z"], nz_lens_4["photo_hists"][:, realization]
zs_k5, nz_k5 = nz_source_0["z"], nz_source_0["photo_hists"][:, realization]
zs_k6, nz_k6 = nz_source_1["z"], nz_source_1["photo_hists"][:, realization]
zs_k7, nz_k7 = nz_source_2["z"], nz_source_2["photo_hists"][:, realization]
zs_k8, nz_k8 = nz_source_3["z"], nz_source_3["photo_hists"][:, realization]
zs_k9, nz_k9 = nz_source_4["z"], nz_source_4["photo_hists"][:, realization]

_nz_lens_0 = Dict("z"=>  zs_k0, "dndz"=>  nz_k0)
_nz_lens_1 = Dict("z"=>  zs_k1, "dndz"=>  nz_k1)
_nz_lens_2 = Dict("z"=>  zs_k2, "dndz"=>  nz_k2)
_nz_lens_3 = Dict("z"=>  zs_k3, "dndz"=>  nz_k3)
_nz_lens_4 = Dict("z"=>  zs_k4, "dndz"=>  nz_k4)
_nz_source_0 = Dict("z"=>  zs_k5, "dndz"=>  nz_k5)
_nz_source_1 = Dict("z"=>  zs_k6, "dndz"=>  nz_k6)
_nz_source_2 = Dict("z"=>  zs_k7, "dndz"=>  nz_k7)
_nz_source_3 = Dict("z"=>  zs_k8, "dndz"=>  nz_k8)
_nz_source_4 = Dict("z"=>  zs_k9, "dndz"=>  nz_k9)

meta, files = make_data(sacc_file, yaml_file;
                            nz_lens_0=_nz_lens_0,
                            nz_lens_1=_nz_lens_1,
                            nz_lens_2=_nz_lens_2,
                            nz_lens_3=_nz_lens_3,
                            nz_lens_4=_nz_lens_4,
                            nz_source_0=_nz_source_0,
                            nz_source_1=_nz_source_1,
                            nz_source_2=_nz_source_2,
                            nz_source_3=_nz_source_3,
                            nz_source_4=_nz_source_4)

meta.types = [ 
    "galaxy_density",
    "galaxy_density",
    "galaxy_density",
    "galaxy_density",
    "galaxy_density",
    "galaxy_shear", 
    "galaxy_shear", 
    "galaxy_shear",
    "galaxy_shear",
    "galaxy_shear"]

scale = 0.03
cov = scale*meta.cov
Γ = sqrt(cov)
iΓ = inv(Γ);

Assuming data rows are in the correct order as it was before version 1.0.
lens_0 lens_0 6
lens_1 lens_1 7
lens_2 lens_2 8
lens_3 lens_3 9
lens_4 lens_4 9
source_0 lens_0 6
source_0 lens_1 7
source_0 lens_2 8
source_0 lens_3 9
source_0 lens_4 9
source_1 lens_0 8
source_1 lens_1 7
source_1 lens_2 8
source_1 lens_3 9
source_1 lens_4 9
source_2 lens_0 8
source_2 lens_1 7
source_2 lens_2 8
source_2 lens_3 9
source_2 lens_4 9
source_3 lens_0 8
source_3 lens_1 7
source_3 lens_2 8
source_3 lens_3 9
source_3 lens_4 9
source_4 lens_0 8
source_4 lens_1 7
source_4 lens_2 8
source_4 lens_3 9
source_4 lens_4 9
source_0 source_0 13
source_1 source_0 13
source_1 source_1 13
source_2 source_0 13
source_2 source_1 13
source_2 source_2 13
source_3 source_0 13
source_3 source_1 13
source_3 source_2 13
source_3 source_3 13
source_4 source_0 13
source_4 source_1 13
source_4 source_2 13
source_4 source_3 13
source_4 source_4 13
using custom nz for nz_source_0
using custom nz for nz_source_1
using custom nz f

### Make Data

In [18]:
function make_theory(;
    Ωm=0.27347, Ωb=0.04217, h=0.71899, σ8=0.779007, ns=0.99651,
    lens_0_b=0.879118, 
    lens_1_b=1.05894, 
    lens_2_b=1.22145, 
    lens_3_b=1.35065, 
    lens_4_b=1.58909,
    dz_lens_0=0.0,
    dz_lens_1=0.0,
    dz_lens_2=0.0,
    dz_lens_3=0.0,
    dz_lens_4=0.0,
    dz_source_0=0.0,
    dz_source_1=0.0,
    dz_source_2=0.0,
    dz_source_3=0.0,
    dz_source_4=0.0,
    A_IA=0.25179439,
    meta=meta, files=files)

    lens_0_zs   = @.(zs_k0 + dz_lens_0)
    lens_1_zs   = @.(zs_k1 + dz_lens_1)
    lens_2_zs   = @.(zs_k2 + dz_lens_2)
    lens_3_zs   = @.(zs_k3 + dz_lens_3)
    lens_4_zs   = @.(zs_k4 + dz_lens_4)
    source_0_zs = @.(zs_k5 + dz_source_0)
    source_1_zs = @.(zs_k6 + dz_source_1)
    source_2_zs = @.(zs_k7 + dz_source_2)
    source_3_zs = @.(zs_k8 + dz_source_3)
    source_4_zs = @.(zs_k9 + dz_source_4)

    nuisances = Dict(
    "lens_0_b"    => lens_0_b,
    "lens_1_b"    => lens_1_b,
    "lens_2_b"    => lens_2_b,
    "lens_3_b"    => lens_3_b,
    "lens_4_b"    => lens_4_b,
    "lens_0_zs"   => lens_0_zs,
    "lens_1_zs"   => lens_1_zs,
    "lens_2_zs"   => lens_2_zs,
    "lens_3_zs"   => lens_3_zs,
    "lens_4_zs"   => lens_4_zs,
    "source_0_zs" => source_0_zs,
    "source_1_zs" => source_1_zs,
    "source_2_zs" => source_2_zs,
    "source_3_zs" => source_3_zs,
    "source_4_zs" => source_4_zs,
    "A_IA"        => A_IA)
       
    cosmology = Cosmology(Ωm=Ωm, Ωb=Ωb, h=h, ns=ns, σ8=σ8,
        tk_mode=:EisHu,
        pk_mode=:Halofit)

    return Theory(cosmology, meta, files; 
             Nuisances=nuisances)
end


fake_data = make_theory();
fake_data = iΓ * fake_data
data = fake_data


@model function model(data)
    Ωm ~ Uniform(0.2, 0.4)
    Ωbb ~ Uniform(0.3, 0.5) # 10*Ωb 
    Ωb = 0.1*Ωbb 
    h ~ Truncated(Normal(0.72, 0.05), 0.64, 0.82)
    σ8 ~ Uniform(0.6, 0.9)
    ns ~ Uniform(0.9, 1.0)

    lens_0_b ~ Uniform(0.75, 0.95)
    lens_1_b ~ Uniform(0.95, 1.15)
    lens_2_b ~ Uniform(1.10, 1.30)
    lens_3_b ~ Uniform(1.20, 1.50)
    lens_4_b ~ Uniform(1.40, 1.80)
    #A_IA ~ Uniform(-1.0, 1.0)

    #dz_lens_0 := (chol_lens_0 * alphas_lens_0)[1]
    #dz_lens_1 := (chol_lens_1 * alphas_lens_1)[1]
    #dz_lens_2 := (chol_lens_2 * alphas_lens_2)[1]
    #dz_lens_3 := (chol_lens_3 * alphas_lens_3)[1]
    #dz_lens_4 := (chol_lens_4 * alphas_lens_4)[1]
    #dz_source_0 := (chol_source_0 * alphas_source_0)[1]
    #dz_source_1 := (chol_source_1 * alphas_source_1)[1]
    #dz_source_2 := (chol_source_2 * alphas_source_2)[1]
    #dz_source_3 := (chol_source_3 * alphas_source_3)[1]
    #dz_source_4 := (chol_source_4 * alphas_source_4)[1]

    theory := make_theory(Ωm=Ωm, Ωb=Ωb, h=h, σ8=σ8, ns=ns,
                            dz_lens_0=dz_0,
                            dz_lens_1=dz_1,
                            dz_lens_2=dz_2,
                            dz_lens_3=dz_3,
                            dz_lens_4=dz_4,
                            dz_source_0=dz_5,
                            dz_source_1=dz_6,
                            dz_source_2=dz_7,
                            dz_source_3=dz_8,
                            dz_source_4=dz_9,
                            lens_0_b=lens_0_b, 
                            lens_1_b=lens_1_b,
                            lens_2_b=lens_2_b, 
                            lens_3_b=lens_3_b,
                            lens_4_b=lens_4_b, 
                            #A_IA=A_IA,
                            )
    ttheory = iΓ * theory
    d = data - ttheory
    Xi2 := dot(d, d)
    data ~ MvNormal(ttheory, I)
end


model (generic function with 2 methods)

In [19]:
cond_model = model(data)

DynamicPPL.Model{typeof(model), (:data,), (), (), Tuple{Vector{Float64}}, Tuple{}, DynamicPPL.DefaultContext}(model, (data = [6.572027413409136, 12.435763191268155, 17.588465216436347, 23.80666087744998, 33.870609952382104, 43.17266338073668, 6.796233019713085, 12.733396279813293, 18.320259204995338, 25.476659702153597  …  6.844577327797751, 8.394187758955358, 8.773474535058208, 20.022452319349235, 50.254763394353645, 58.53653690290034, 65.46888046081511, 71.90186190488987, 76.5605830019243, 77.83792181295414],), NamedTuple(), DynamicPPL.DefaultContext())

In [20]:
using OptimizationOptimJL: NelderMead
map = maximum_a_posteriori(cond_model, NelderMead())

ModeResult with maximized lp of -410.29
[0.2875285932448112, 0.4998869724194598, 0.7250203514899972, 0.7590935355201506, 0.9992570544175188, 0.9079362200841538, 1.0922620270179029, 1.2559804914787256, 1.3883845372201646, 1.6361465538695703  …  6.5802741935043e-9, 4.227447643843176e-9, 2.6892480515736377e-9, 1.7193308496094545e-9, 1.1246313064302255e-9, 7.553750936256795e-10, 5.158565172931828e-10, 3.5244272148088273e-10, 2.373492775558387e-10, 49.07104191908309]

In [35]:
values = map.values.array
namess = map.values.dicts[1].keys

448-element Vector{Symbol}:
 :Ωm
 :Ωbb
 :h
 :σ8
 :ns
 :lens_0_b
 :lens_1_b
 :lens_2_b
 :lens_3_b
 :lens_4_b
 Symbol("theory[1]")
 Symbol("theory[2]")
 Symbol("theory[3]")
 ⋮
 Symbol("theory[427]")
 Symbol("theory[428]")
 Symbol("theory[429]")
 Symbol("theory[430]")
 Symbol("theory[431]")
 Symbol("theory[432]")
 Symbol("theory[433]")
 Symbol("theory[434]")
 Symbol("theory[435]")
 Symbol("theory[436]")
 Symbol("theory[437]")
 :Xi2

In [37]:
params = Dict{Symbol, Float64}()
for (i, name) in enumerate(namess)
    params[Symbol(name)] = values[i]
end

In [38]:
 CSV.write(joinpath(folpath, filename), DataFrame(params))

LoadError: UndefVarError: `folpath` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
typeof(mle_estimate.values)

In [ ]:
CSV.write(mle_estimate.values.array, "mle_estimate.csv")

In [ ]:
plot(meta_gcgc.data, yscale=:log10, label="Data - gcgc")
plot!(t_gcgc, label="Theory - gcgc")

In [ ]:
break

In [ ]:
npzwrite("data/CosmoDC2/theories.npz", 
    t_wlwl=t_wlwl,
    t_gcgc=t_gcgc,
    t_3x2=t_3x2)

In [ ]:
A_IAs = Vector(range(-1.0, 1.0, length=200));

In [ ]:
A_IAs = range(-1.0, 1.0, length=200)
IA_xi2_3x2 = [model_3x2(A_IA=A_IA, data=t_3x2)[2] for A_IA in A_IAs]
IA_xi2_wlwl = [model_wlwl(A_IA=A_IA, data=t_wlwl)[2] for A_IA in A_IAs];

In [ ]:
plot(A_IAs, IA_xi2_wlwl, label="wlwl")
plot!(A_IAs, IA_xi2_3x2, label="3x2")

In [ ]:
dzs = zeros(5)
wzs = ones(5)
es = LinRange(-0.2, 0.2, 200)
dzs_xi2_3x2 = [model_3x2(dzs_lens=dzs .+ e, dzs_source=dzs .+ e, data=t_3x2)[2] for e in es]
dzs_xi2_gcgc = [model_gcgc(dzs_lens=dzs .+ e, data=t_gcgc)[2] for e in es]
dzs_xi2_wlwl = [model_wlwl(dzs_source=dzs .+ e, data=t_wlwl)[2] for e in es]

In [ ]:
plot(es, dzs_xi2_gcgc, label="gcgc")
plot!(es, dzs_xi2_wlwl, label="wlwl")
plot!(es, dzs_xi2_3x2, label="3x2")

In [ ]:
wzs_xi2_3x2 = [model_3x2(wzs_lens=wzs .+ e, wzs_source=wzs .+ e, data=t_3x2)[2] for e in es]
wzs_xi2_gcgc = [model_gcgc(wzs_lens=wzs .+ e, data=t_gcgc)[2] for e in es]
wzs_xi2_wlwl = [model_wlwl(wzs_source=wzs .+ e, data=t_wlwl)[2] for e in es]

In [ ]:
plot(es, wzs_xi2_gcgc, label="gcgc", yscale = :log10)
plot!(es, wzs_xi2_wlwl, label="wlwl")
plot!(es, wzs_xi2_3x2, label="3x2")

In [ ]:
#using ForwardDiff
#function grad_h(h)
#    function f(h)
#        t, Xi2 = model_3x2(;Ωm=0.27347, σ8=0.779007, Ωb=0.04217, h=h, ns=0.99651,
#            meta=meta_3x2, files=files_3x2, data=t_3x2)
#        return Xi2
#    end
#    return ForwardDiff.derivative(f, h)
#end

#function grad_Wm(Wm)
#    function f(Wm)
#        t, Xi2 = model_3x2(;Ωm=Wm, σ8=0.779007, Ωb=0.04217, h=0.71899, ns=0.99651,
#            meta=meta_3x2, files=files_3x2, data=t_3x2)
#        return Xi2
#    end
#    return ForwardDiff.derivative(f, Wm)
#end

In [ ]:
Wm = range(0.2, 0.5, length=200)
Wb = range(0.03, 0.07, length=200)
h  = range(0.6, 0.9, length=200)
s8 = range(0.4, 1.2, length=200)
ns = range(0.84, 1.1, length=200)

In [ ]:
#Wm_grads = [grad_Wm(Wm_i) for Wm_i in Wm]
Wm_xi2_3x2 = [model_3x2(;Ωm=Wm_i, data=t_3x2)[2] for Wm_i in Wm]
Wm_xi2_gcgc = [model_gcgc(;Ωm=Wm_i, data=t_gcgc)[2] for Wm_i in Wm]
Wm_xi2_wlwl = [model_wlwl(;Ωm=Wm_i, data=t_wlwl)[2] for Wm_i in Wm];

In [ ]:
#Wb_grads = [grad_Wm(Wm_i) for Wm_i in Wm]
Wb_xi2_3x2 = [model_3x2(;Ωb=Wb_i, data=t_3x2)[2] for Wb_i in Wb]
Wb_xi2_gcgc = [model_gcgc(;Ωb=Wb_i, data=t_gcgc)[2] for Wb_i in Wb]
Wb_xi2_wlwl = [model_wlwl(;Ωb=Wb_i, data=t_wlwl)[2] for Wb_i in Wb];

In [ ]:
#h_grads = [grad_h(h_i) for h_i in h]
h_xi2_3x2 = [model_3x2(;h=h_i, data=t_3x2)[2] for h_i in h]
h_xi2_gcgc = [model_gcgc(;h=h_i, data=t_gcgc)[2] for h_i in h]
h_xi2_wlwl = [model_wlwl(;h=h_i, data=t_wlwl)[2] for h_i in h];

In [ ]:
#s8_grads = [grad_Wm(s8_i) for Wm_i in Wm]
s8_xi2_3x2 = [model_3x2(;σ8=s8_i, data=t_3x2)[2] for s8_i in s8]
s8_xi2_gcgc = [model_gcgc(;σ8=s8_i, data=t_gcgc)[2] for s8_i in s8]
s8_xi2_wlwl = [model_wlwl(;σ8=s8_i, data=t_wlwl)[2] for s8_i in s8];

In [ ]:
#ns_grads = [grad_Wm(ns_i) for ns_i in ns]
ns_xi2_3x2 = [model_3x2(;ns=ns_i, data=t_3x2)[2] for ns_i in ns]
ns_xi2_gcgc = [model_gcgc(;ns=ns_i, data=t_gcgc)[2] for ns_i in ns]
ns_xi2_wlwl = [model_wlwl(;ns=ns_i, data=t_wlwl)[2] for ns_i in ns];

In [ ]:
plot(h, h_xi2_3x2, label="h Xi2s - 3x2", yscale = :log10)
plot!(h, h_xi2_wlwl, label="h Xi2s - wlwl")
plot!(h, h_xi2_gcgc, label="h Xi2s - gcgc")

In [ ]:
plot(Wm, Wm_xi2_3x2, label="Wm Xi2s - 3x2", yscale = :log10)
plot!(Wm, Wm_xi2_wlwl, label="Wm Xi2s - wlwl")
plot!(Wm, Wm_xi2_gcgc, label="Wm Xi2s - gcgc")

In [ ]:
npzwrite("data/CosmoDC2/gradients.npz",
    h=h,
    Wm=Wm,
    Wb=Wb,
    ns=ns,
    s8=s8, 
    Wm_xi2_3x2=Wm_xi2_3x2,
    Wm_xi2_gcgc=Wm_xi2_gcgc,
    Wm_xi2_wlwl=Wm_xi2_wlwl,
    Wb_xi2_3x2=Wb_xi2_3x2,
    Wb_xi2_gcgc=Wb_xi2_gcgc,
    Wb_xi2_wlwl=Wb_xi2_wlwl,
    h_xi2_3x2=h_xi2_3x2,
    h_xi2_gcgc=h_xi2_gcgc,
    h_xi2_wlwl=h_xi2_wlwl,
    s8_xi2_3x2=s8_xi2_3x2,
    s8_xi2_gcgc=s8_xi2_gcgc,
    s8_xi2_wlwl=s8_xi2_wlwl,
    ns_xi2_3x2=ns_xi2_3x2,
    ns_xi2_gcgc=ns_xi2_gcgc,
    ns_xi2_wlwl=ns_xi2_wlwl)